<a href="https://colab.research.google.com/github/divya-gh/ConvoSense-AI-Banking-Chatbot-Analytics/blob/main/notebooks/Linguistic_%26_NLP_Error_Analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### Objective :
- ✔ Isolate misclassified utterances
- ✔ Tokenize text
- ✔ Apply POS tagging
- POS Frequency Analysis
- ✔ Identify syntactic & semantic patterns causing failure
- ✔ Recommend model improvements

### ➡️ This answers:
- “Why is the model failing, linguistically?”

### 🧠 Why?
#### This phase demonstrates:
    - Linguistic reasoning (syntax, semantics, ambiguity)
    - Applied NLP (tokenization, POS, entities)
    - Error-driven model improvement
    - AI governance & explainability

#### Misclassified banking queries often include complex noun phrases (such as “credit card payment reversal”) and modal verbs (“can I,” “should I”), which increase semantic ambiguity. The model also lacks robust entity disambiguation for different transaction types, causing it to misinterpret intent and default to fallback or escalation.

In [8]:
import sys
print(sys.executable)


/usr/bin/python3


In [9]:
!pip install -U spacy
!python -m spacy download en_core_web_sm


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.8/12.8 MB 65.3 MB/s eta 0:00:00
✔ Download and installation successful
You can now load the package via spacy.load('en_core_web_sm')
⚠ Restart to reload dependencies
If you are in a Jupyter or Colab notebook, you may need to restart Python in
order to load all the package's dependencies. You can do this by selecting the
'Restart kernel' or 'Restart runtime' option.


In [1]:
import spacy
nlp = spacy.load("en_core_web_sm")


In [2]:
import os
os.getcwd()


'/content'

In [3]:
import json
import pandas as pd
from pathlib import Path

In [4]:
data_path = Path("./ConvoSense-AI-Banking-Chatbot-Analytics/data/raw_conversations/banking_conversations.json")

with open(data_path, "r") as f:
    conversations = json.load(f)

df = pd.DataFrame(conversations)
df.head()

,conversation_id,timestamp,channel,user_utterance,true_intent,predicted_intent,confidence_score,entities,fallback_triggered,escalated_to_agent,resolved
0,conv_001,2025-01-05T10:15:00,chat,What is my checking account balance?,Check_Account_Balance,Check_Account_Balance,0.92,{'account_type': 'checking'},False,False,True
1,conv_002,2025-01-05T10:18:00,chat,I see a charge I don't recognize from Amazon,Dispute_Transaction,Transaction_History,0.61,{'merchant_name': 'Amazon'},False,True,False
2,conv_003,2025-01-05T10:22:00,chat,My debit card was stolen,Card_Lost_Or_Stolen,Card_Lost_Or_Stolen,0.95,{},False,False,True
3,conv_004,2025-01-05T10:30:00,voice,Can you tell me when my last five transactions...,Transaction_History,Transaction_History,0.88,{'transaction_count': 5},False,False,True
4,conv_005,2025-01-05T10:35:00,chat,I need help updating my phone number,Update_Personal_Details,Default_Fallback,0.42,{'field': 'phone_number'},True,True,False


### Step 1 — Isolate Misclassified Utterances


In [5]:
misclassified = df[df['true_intent'] != df['predicted_intent']]
misclassified[['user_utterance', 'true_intent', 'predicted_intent']].head()



,user_utterance,true_intent,predicted_intent
1,I see a charge I don't recognize from Amazon,Dispute_Transaction,Transaction_History
4,I need help updating my phone number,Update_Personal_Details,Default_Fallback


#### 🔹 Step 2: Tokenization
- Tokenization breaks utterances into meaningful units (words/tokens).

##### 📌 Why this matters
- Long or compound utterances → higher intent confusion
- Banking queries often combine multiple intents in one sentence

In [6]:
def tokenize(text):
    doc = nlp(text)
    return [token.text for token in doc]

misclassified['tokens'] = misclassified['user_utterance'].apply(tokenize)
misclassified[['user_utterance', 'tokens']].head()


/tmp/ipython-input-2712219272.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  misclassified['tokens'] = misclassified['user_utterance'].apply(tokenize)


,user_utterance,tokens
1,I see a charge I don't recognize from Amazon,"[I, see, a, charge, I, do, n't, recognize, fro..."
4,I need help updating my phone number,"[I, need, help, updating, my, phone, number]"


### 🔍 What to Observe (Important)

#### Look for:
- Long token lists
- Conjunctions: and, or
- Modal verbs: can, should, would
- Banking-specific nouns: charge, payment, transfer, balance
- These often signal multi-intent confusion.

🔹 Step 3: POS Tagging

POS (Part-of-Speech) tagging tells us how language structure impacts failure.

In [7]:
def pos_tags(text):
    doc = nlp(text)
    return [(token.text, token.pos_) for token in doc]

misclassified['pos_tags'] = misclassified['user_utterance'].apply(pos_tags)
misclassified[['user_utterance', 'pos_tags']].head()


/tmp/ipython-input-238929192.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  misclassified['pos_tags'] = misclassified['user_utterance'].apply(pos_tags)


,user_utterance,pos_tags
1,I see a charge I don't recognize from Amazon,"[(I, PRON), (see, VERB), (a, DET), (charge, NO..."
4,I need help updating my phone number,"[(I, PRON), (need, VERB), (help, VERB), (updat..."


🔍 What Patterns to Look For:

### Pattern:
- Multiple VERBs
- Nouns chained together
- WH-words (what, why, how)
- Prepositions

### Why It Causes Errors:
 - Multiple user actions
 - Entity ambiguity
 - Open-ended intent
 - Context *switching*

## 🔹 Step 4: Linguistic Error Analysis (Interpretation)
### Linguistic Error Analysis:

#### Analysis of misclassified utterances reveals several recurring linguistic patterns:

• Queries often contain multiple verbs and conjunctions, indicating compound or multi-intent requests that the model is not designed to handle.

• Ambiguous noun phrases (e.g., "charge", "payment", "transfer") lack sufficient contextual entity resolution, leading to incorrect intent classification.

• The frequent use of modal verbs and open-ended question structures increases semantic uncertainty, triggering fallback or escalation.

These findings suggest a need for improved intent decomposition, enhanced entity extraction, and clearer conversational call flows.


## Step 5: Recommendations

### Model Improvement Recommendations

• Introducing hierarchical intent classification to handle multi-action queries.

• Expand entity taxonomy for transaction-related concepts: give the model more detailed categories for transaction types so it can understand customer queries more accurately

• Add clarification prompts before escalation to reduce unnecessary handoffs.

• Retrain the model using linguistically diverse utterances including compound sentences.

• Implement confidence-based routing to optimize customer experience and operational cost.


## What “entity taxonomy” ?
An entity taxonomy is a structured list of categories the model uses to recognize and classify things.
In banking, entities might include:
- Payment
- Refund
- Reversal
- Chargeback
- Transfer
- Deposit
- vWithdrawal

Right now, model probably has only a few broad categories, which makes it easy for it to confuse similar concepts.

What “expanding” means here
we would add more specific, fine‑grained labels so the model can distinguish between transaction types that look similar but mean different things.

### For example:

#### Broad Entity:Expanded Entities
- Payment: Bill payment, credit card payment, loan payment
- Refund: Merchant refund, partial refund, auto‑refund
- Reversal: ACH reversal, credit card reversal
- Transfer: Internal transfer, external transfer, wire transfer

#### Why this helps ?
When users write things like:
- “credit card payment reversal”
- “can I dispute this transaction”
- “should I request a refund”

the model needs a richer vocabulary of transaction‑related entities to correctly interpret intent.

A more detailed taxonomy reduces ambiguity and prevents misclassification.